<h1 align="center">Real Estate Price Prediction</h1>

**This short Notebook correspond to the score 0.16643**

<h1 align="center">LOADING PIPELINE</h1>

In [1]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

!pip install auto-sklearn
!pip install scipy
!pip install seaborn
!pip install geopy
!pip install haversine
!pip install lightgbm
!pip install xgboost
!pip install pipelineprofiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   249  100   249    0     0    294      0 --:--:-- --:--:-- --:--:--   294
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionPars

Data storage :

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We are using external data in addition to the data provided in kaggle, this links contains a folder DATA with all csv files needed to reproduce our prediction (the folder is also provided on BLACKBOARD as a zip file):

https://drive.google.com/drive/folders/1-4z5G7FYxyJrqBjG9cHiiBi5Z2ZjuNVV

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno  # For missing data visualization
from google.colab import files
from geopy.distance import distance
from haversine import haversine, haversine_vector, Unit


from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb

import autosklearn.metrics
import autosklearn.pipeline.components.regression
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.pipeline.components.base import AutoSklearnRegressionAlgorithm

import PipelineProfiler   # For auto-sklearn model interpretation  

np.random.seed(123)
sns.set_style('darkgrid')
pd.set_option('display.max_colwidth', None)


# We are using a folder data provided by link and on blackboard
data_filepath = './drive/MyDrive/data/' 
pd.options.display.max_columns = None # Be careful using this with one-hot encoding

In [3]:
apartments = pd.read_csv(data_filepath + 'apartments_train.csv')


In [4]:
buildings = pd.read_csv(data_filepath + 'buildings_train.csv')


Merge apartments and buildings in the same dataframe :

In [5]:

data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

In [6]:
apartments_test = pd.read_csv(data_filepath + 'apartments_test.csv')
buildings_test = pd.read_csv(data_filepath + 'buildings_test.csv')

data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)


In [8]:
ap_test_id = apartments_test['id']
ap_bu_train_id = apartments['building_id']
ap_bu_test_id = apartments_test['building_id']
bu_train_id = buildings['id']
bu_test_id = buildings_test['id']

n_ap_train = apartments.shape[0]
n_ap_test = apartments_test.shape[0]
n_bu_train = buildings.shape[0]
n_bu_test = buildings_test.shape[0]

y_train = apartments.price.values

all_ap = pd.concat((apartments, apartments_test)).reset_index(drop=True)
all_bu = pd.concat((buildings, buildings_test)).reset_index(drop=True)

In [9]:
all_data = pd.merge(all_ap, all_bu.set_index('id'), how='left', left_on='building_id', right_index=True)


<h1 align="center">PREPROCESSING PIPELINE</h1>

In [10]:
all_ap.drop(['price'], axis=1, inplace=True)

Fill some obvious 

In [11]:
all_bu['latitude'] = all_bu['latitude'].fillna(55.5675811)
all_bu['longitude'] = all_bu['longitude'].fillna(37.48152684)

In [12]:
all_bu['elevator_without'] = all_bu['elevator_without'].replace(np.NaN, all_bu['elevator_without'].median())

In [13]:
all_data = pd.merge(all_ap, all_bu.set_index('id'), how='left', left_on='building_id', right_index=True)

all_data.drop('id', axis=1, inplace=True)
all_data.drop('building_id', axis=1, inplace=True)

In [14]:
street_dict = {}
def convert_to_int(street):
    if street in street_dict: return street_dict[street]
    integer = len(street_dict.keys())
    street_dict[street] = integer
    return integer

In [15]:
all_bu['ordinal_street'] = np.array([convert_to_int(word) for word in all_bu['street']])  # Useful for feature engineering, i.e. groupby()

Correct some obvious wrong values for the location based on the real location on google maps :

In [16]:
all_bu.at[6973, 'latitude'] = 55.63384636810666
all_bu.at[6973, 'longitude'] = 37.41986901627135

all_bu.at[7899, 'latitude'] = 55.54371958666631
all_bu.at[7899, 'longitude'] = 37.48233889847151

all_bu.at[9061, 'latitude'] = 55.62773817227544
all_bu.at[9061, 'longitude'] = 37.46499797519564

all_bu.at[9512, 'latitude'] = 55.80920516603121
all_bu.at[9512, 'longitude'] = 37.34990449848708

all_bu.at[9629, 'latitude'] = 55.54371958666631
all_bu.at[9629, 'longitude'] = 37.48233889847151

**FEATURE GENERATION**

Create new useful features such as the distance to points of interests (e.g, train tram or subways stations or park to relax)

In [17]:
all_bu['coordinates'] = list(zip(all_bu['latitude'], all_bu['longitude']))

def get_min_distance(feature_name):
  feature = pd.read_csv(f'{data_filepath}{feature_name}.csv', dtype='float64')
  feature['coordinates'] = list(zip(feature['latitude'], feature['longitude']))

  bu_coord = all_bu['coordinates'].to_numpy(copy=True).tolist()
  feature_coord = feature['coordinates'].to_numpy(copy=True).tolist()

  distances = haversine_vector(bu_coord, feature_coord, unit=Unit.KILOMETERS, comb=True)

  distances_frame = pd.DataFrame(data=distances)
  feature_min = pd.DataFrame({f'{feature_name}_min': distances_frame.min(axis=0)})
  feature_min.to_csv(f'{data_filepath}{feature_name}_min.csv', index=False)
  all_bu[f'{feature_name}_min'] = feature_min

get_min_distance('subway')
get_min_distance('train')
get_min_distance('tram')
get_min_distance('park')


New feature transit: minimum distance to a facility/infrastructure

In [18]:
all_bu['transit_min'] = all_bu[['subway_min', 'train_min', 'tram_min']].min(axis=1)

New feature radius : the distance from an apartment to the "center" of the city

In [19]:
latitude_mean = all_bu['latitude'].mean(axis=0, skipna=True)
longitude_mean = all_bu['longitude'].mean(axis=0, skipna=True)
latitude_kremlin = 55.75202223    # Dist, to kremlin yields lower correlation than mean
longitude_kremlin = 37.61749837

difference_latitude = all_bu['latitude'] - latitude_mean
difference_longitude = all_bu['longitude'] - longitude_mean
radius = pd.DataFrame({"radius":np.sqrt(np.square(difference_latitude) + np.square(difference_longitude))})

middle = (latitude_mean, longitude_mean)

In [20]:
radius = all_bu['coordinates'].apply(lambda x: distance(x, middle))


In [21]:
radius = radius.astype(str).str[:-3].astype(float)

In [22]:
all_bu['radius'] = pd.DataFrame({"radius":radius})
all_bu['closeness'] = 1 / np.sqrt(radius)


The data engineering has been made for some features related to the buildings (the distance for a subway station is the same for all the apartments in the same building), now It will be done for features directly related to the apartments : 

In [23]:
# Getting new test and train sets:

apartments = all_ap[:n_ap_train]
apartments['price'] = y_train         # train set
# apartments.drop(index=2804, inplace=True)
# apartments.drop(index=15840, inplace=True)
# apartments.drop(index=21414, inplace=True)
apartments_test = all_ap[n_ap_train:] # number in train

buildings = all_bu[:n_bu_train]
buildings_test = all_bu[n_bu_train:]

data = pd.merge(apartments, buildings.set_index('id'), how='inner', left_on='building_id', right_index=True)
data_y = data['price']
data_x = data.drop(['price'], axis=1)

data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='inner', left_on='building_id', right_index=True)

n_train = data.shape[0]
all_data = pd.concat((data, data_test)).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
compressed_floor = all_data.groupby(['building_id'])['floor'].transform(lambda x: x - x.min())
compressed_stories = all_data.groupby(['building_id'])['floor'].transform(lambda x: x.max())

In [25]:
all_data['stairs_to_walk'] = pd.DataFrame({"stairs_to_walk": 1 / ((1 - all_data['elevator_without'])* (compressed_stories - compressed_floor) + 1)})

In [26]:
all_data['kegness'] = all_data['area_total'] / all_data['radius']

In [27]:
all_data['seller'] = all_data['seller'].astype('category')
all_data['layout'] = all_data['layout'].astype('category')
all_data['condition'] = all_data['condition'].astype('category')
all_data['district'] = all_data['district'].astype('category')
all_data['street'] = all_data['street'].astype('category')
all_data['address'] = all_data['address'].astype('category')
all_data['material'] = all_data['material'].astype('category')
all_data['parking'] = all_data['parking'].astype('category')
all_data['heating'] = all_data['heating'].astype('category')

In [28]:
data = all_data[:n_train]
data_test = all_data[n_train:]

In [29]:
final_train = data.drop(['id', 'building_id', 'street', 'ordinal_street', 'address', 'coordinates', 'tram_min', 'train_min', 'subway_min'], axis=1) # 'windows_court', 'phones', 'new'
final_test = data_test.drop(['id', 'building_id', 'street', 'ordinal_street', 'address', 'coordinates', 'tram_min', 'train_min', 'subway_min'], axis=1) # 'windows_court', 'phones', 'new'

Our features are one-hot encoded 

In [30]:
one_hot_encoded_training_predictors = pd.get_dummies(final_train)
one_hot_encoded_test_predictors = pd.get_dummies(final_test)

final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='inner', 
                                                                    axis=1)
final_test.drop(['price'], axis=1, inplace=True)

Then the target is log transformed

In [31]:
final_train['price'] = np.log(final_train['price'])
data_y = np.log(data_y)

We drop the price since it's our target

In [32]:
final_train.drop(['price'], axis=1, inplace=True)

<h1 align="center">MODEL FITTING</h1>

Our best model was achieved using the following configuration #


*   A training time of 2 hours
*   RMSE as the metric to optimize since the target is log transformed
*   Cross-validation with a fold of 5 
*   Default parameters of the AutoSklearnRegressor method : we are training an ensemble of 50 differents models and fine-tuning is handled by the library
*   Moreover, the imputation of missing values is included in the method



In [33]:
training_time = 60 * 60 * 2

In [34]:
auto_regr = AutoSklearnRegressor(time_left_for_this_task= training_time, 
                                 metric=autosklearn.metrics.root_mean_squared_error,
                                 resampling_strategy='cv',
                                 resampling_strategy_arguments={'folds':5},
                                 seed=41)


In [35]:
auto_regr.fit(final_train, data_y, dataset_name='Moscow Housing Kaggle')

AutoSklearnRegressor(metric=root_mean_squared_error, per_run_time_limit=720,
                     resampling_strategy='cv',
                     resampling_strategy_arguments={'folds': 5}, seed=41,
                     time_left_for_this_task=7200)

In [36]:
auto_regr_predictions = auto_regr.predict(final_test)
auto_regr_predictions = np.exp(auto_regr_predictions)

In [37]:
prediction_frame = pd.DataFrame(columns={'id': data_test['id'], 'price_prediction': auto_regr_predictions})
prediction_frame['id'] = data_test['id']
prediction_frame['price_prediction'] = auto_regr_predictions
prediction_frame.to_csv(f'{data_filepath}prediction/finalprediction_{training_time / 60}_mins.csv', index=False)

In [ ]:
files.download(f'{data_filepath}prediction/finalprediction_{training_time / 60}_mins.csv')